# 09 추천 시스템
## 최근접 이웃 협업 필터링

친구들에게 물어보는 것과 유사한 방식으로, 상요자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식(User Behavior)만을 기반으로 추천을 수행하는 것이 협업 필터링(Collaborative filtering)방식이다.

#### 협업 필터링의 유형
+ 최근접 이웃 기반(Nearest Neighbor)
  + 사용자 기반(User-user CF)
  + 아이템 기반(item-item CF)
+ 잠재 요인 기반(Latent Factor)
  + 행렬 분해 기반(Matrix Factorization)

#### 협업 필터링의 특징
+ User behavior (item 구매 이력, 영화 평점 이력)에만 기반하여 추천 알고리즘 들을 전박적으로 지칭함
+ 상품, 영화 등 사용자가 아직 평가하지 않은 item에 대한 평가(rating)을 예측하는 것이 주요 역할
![](https://velog.velcdn.com/images%2Fgjtang%2Fpost%2F8e54b423-3231-4317-88ec-38d8c012494b%2Fimage.png)

#### 협업 필터링을 위한 데이터 세트 - 사용자 로우 -아이템 컬럼
![](https://velog.velcdn.com/images%2Fgjtang%2Fpost%2Fe1dfbf88-08b4-4fea-a7bd-2acdeca27bc1%2Fimage.png)

#### 사용자 기반과 아이템 기반 협업 필터링 이해
+ 사용자 기반(User-User)
  + 특정 사용자와 비슷한 고객들을 기반으로 이 비슷한 고객들이 선호하는 다른 상품을 추천
  + 특정 사용자와 비슷한 상품을 구매해온 고객들은 비슷한 고객으로 간주
  + ex) 당신과 비슷한 고객들이 다음 상품도 구매했습니다.
   ![](https://velog.velcdn.com/images%2Fgjtang%2Fpost%2Fbd4ad44a-cd30-49d5-890e-415506c33b49%2Fimage.png)
+ 아이템 기반(item-item)
  + 특정 상품과 유사한 좋은 평가를 받은 다른 비슷한 상품을 추천
  + 사용자들로부터 특정 상품과 비슷한 평가를 받은 상품들은 비슷한 상품으로 간주
  + ex) 이 상품을 선택한 다른 고객들은 다음 상품도 구매했습니다.
    ![](https://velog.velcdn.com/images%2Fgjtang%2Fpost%2F2dcbb40a-7a14-47f6-8503-1c9bbb3dfc8d%2Fimage.png)

#### 사용자 기반 vs 아이템 기반

일반적으로 사용자 기반 보다는 아이템 기반 방식이 더 선호된다. 사람간의 특성은 상대적으로 다양한 요소들에 기반한다. 단순히 동일한 상품을 구입하였다고 유사한 사람이라고 판단하기 어려운 경우가 많기 때문이다.

#### 협업 필터링을 위한 코사인 유사도

유사도 측정 방법인 코사인 유사도는 추천 시스템의 유사도 측정에 가장 많이 적용된다. 추천 시스템에 사용되는 데이터는 피처 벡터화된 텍스트 데이터와 동일하게 다차원 회소 행렬이라는 특징이 있으므로 유사도 측정을 위해 주로 코사인 유사도를 이용한다.

![](https://velog.velcdn.com/images%2Fgjtang%2Fpost%2F812090da-dcad-44f0-bc0a-9380b11840b4%2Fimage.png)

#### 아이템 기반 협업 필터링의 개인화된 영화 추천
사용자 u의 아이템 i에 대한 평점 예측을 사용자 u가 아이템 i와 유사한 다른 아이템들의 합으로 계산하되, 아이템 i와 다른 아이템들간의 유사도를 반영한 합으로 계산한다.

![](https://velog.velcdn.com/images%2Fgjtang%2Fpost%2Fd45684e1-9470-456e-8000-e97adf2c2a7b%2Fimage.png)

#### 아이템 기반의 협업 필터링에서 개인화된 예측 평점
$\hat{R}_{u,i} = \sum ^{N} (S_{i,N} * R_{u,N}) / \sum ^N (|S_{i,N}|)$

+ $\hat{R}_{u,i}$ : 사용자 u, 아이템 i 의 개인화된 예측 평점 값
+ $S_{i,N}$ : 아이템 i와 가장 유사도가 높은 Top-N개 아이템의 유사도 벡터
+ $R_{u,N}$ : 사용자 u의 아이템 i와 가장 유사도가 높은 Top-N개 아이템에 대한 실제 평점 벡터

#### 아이템 기반 협업필터링 구현순서
1. 사용자-아이템 행렬 데이터를 아이템-사용자 행렬 데이터로 변환
2. 아이템 간의 코사인 유사도로 아이템 유사도 산출
3. 사용자가 관람하지 않은 아이템들 중에서 아이템간 유사도를 반영한 예측 점수 계산
4. 예측 점수가 가장 높은 순으로 아이템 추천

## 06 아이템 기반 최근접 이웃 협업 필터링 실습

일반적으로 추천 정확돠 더 뛰어난 아이템 기반의 협업 필터링을 구현해본다. Grouplens 사이트에서 만든 MovieLens 데이터를 이용해 실습하도록 한다.

https://grouplens.org/datasets/movielens/latest/

#### 데이터 가공 및 변환

In [1]:
import pandas as pd
import numpy as np

movies = pd.read_csv('./ml-latest-small/movies.csv')
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
print(movies.shape)
print(ratings.shape)

(9742, 3)
(100836, 4)


In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


+ movies.csv : 영화에 대한 메타 정보인 title과 genres를 가지고 있는 영화 정보
+ ratings.csv : 사용자별로 영화에 대한 평점을 매긴 데이터 세트

협업 필터링은 이 ratings.csv 데이터 세트와 같이 사용자와 아이템 간의 평점(또는 다른 유형의 액션)에 기반해 추천하는 시스템이다. 이를 이용해 아이템 기반의 최근접 이웃 협업 필터링을 구현해 본다. 

먼저 로우 레벨 형태의 원본 데이터 세트를 모든 사용자를 로우로, 모든 영화를 칼럼으로 구성한 데이터 세트로 변경해야 한다.

In [4]:
ratings = ratings[['userId', 'movieId', 'rating']]
# pivot_table() 함수를 이용하여 로우레벨의 값을 칼럼으로 변경
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


NaN 값이 많은 이유는 사용자가 아직 평점을 매기지 않응 영화가 칼럼으로 변환되면서 NaN으로 값이 할당됐기 때문이다.

In [5]:
# 영화명은 movies 데이터세트에 존재
# 영화명 (title) 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# 가독성을 위해 columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# 최소평점이 0.5점이므로 NaN 값을 모두 0 으로 변환
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 유사도 산출

변환된 사용자-영화 평점 행렬 데이터 세트를 이용해 영화 간의 유사도를 측정한다. 영화간의 유사도는 코사인 유사도를 기반으로 한다.

영화를 기준으로 cosine_similarity()를 적용하려면 현재의 ratings_matrix가 행 기준이 영화가 되고 열 기준이 사용자가 되어야 한다.

In [6]:
# transpose()를 사용하여 행과 열을 바꾼다.
ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


ratings_matrix를 전치 행렬 형식으로 변경한 데이터 세트를 기반으로 영화의 코사인 유사도를 구해본다. 좀 더 직관적인 영화의 유사도 값을 표현하기 위해 cosine_similarity()로 반환된 넘파이 행렬에 영화명을 매핑해 DataFrame으로 변환한다.

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# cosine_similarity() 로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns,
                          columns=ratings_matrix.columns)
print(item_sim_df.shape)
item_sim_df.head(3)

(9719, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


영화의 유사도 행렬인 item_sim이 생성됐다. item_sim을 DataFrame으로 변환한 item_sim_df를 이용해 영화 '대부'와 유사도가 높은 상위 6개 영화를 추출해 본다.

In [8]:
item_sim_df["Godfather, The (1972)"].sort_values(ascending=False)[:6]

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

기준 영화인 '대부'를 제외하면 '대부-2편'이 가장 유사도가 높다. 그 다음은 마틴 스콜세지 감독의 '좋은 친구들'이다. 앞의 콘텐츠 기반 필터링과 다른 점은 '뻐꾸기 둥지 위로 날아간 새', '스타워즈 1편'과 같이 장르가 완전히 다른 영화도 유사도가 매우 높게 나타났다는 것이다.

이번에는 다른 훌륭한 영화인 '인셉션'과 유사도가 높은 영화를 찾아본다. '인셉션' 자신은 유사도에서 제외한다.

In [9]:
item_sim_df["Inception (2010)"].sort_values(ascending=False)[1:6]

title
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

'다크나이트'가 가장 유사도가 높다. 그 다음은 주로 스릴러와 액션이 가미된 좋은 영화가 높은 유사도를 나타내고 있다. 만들어진 아이템 기반 유사도 데이터는 사용자의 평점 정보를 모두 취합해 영화에 따라 유사한 다른 영화를 추천할 수 있게 해준다.

이번에는 아이템 기반 유사도 데이터를 이용해 개인에게 특화된(Personalized) 영화 추천 알고리즘을 만들어본다.

#### 아이템 기반 최근점 이욳 협업 필터링으로 개인화된 영화 추천

앞 예제에서 생성된 영화 간의 유사도를 가지는 DataFrame인 item_sim_df와 사용자-영화 평점 DataFrame인 ratings_matrix 변수를 계속 활용해 사용자별로 최적화된 평점 스코어 ($\hat{R}_{u, i}$) 를 예측하는 함수를 만든다.

$\hat{R}_{u, i} = \sum (S_{i,N} * R_{u,N})/ \sum^N(|S_{i, N}|)$

In [10]:
def predict_rating(ratings_arr, item_sim_arr ):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

위 코드에서 ratings_arr.dot(item_sim_arr)은 $\sum (S_{i,N} * R_{u,N})$ 를 계산한 값이다. np.array([np.abs(item_sim_arr).sum(axis=1)])는 $\sum^N(|S_{i, N}|)$를 계산한 값이다.

predict_rating() 함수를 이용해 개인화된 예측 평점을 구해본다.

In [11]:
ratings_pred = predict_rating(ratings_matrix.values , item_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


예측 평점이 사용자별 영화의 실제 평점과 영화의 코사인 유사도를 내적한 값이기 때문에 기존에 영화를 관람하지 않아 0에 해당했던 실제 영화 평점이 예측에서는 값이 부여되는 경우가 많이 발생했다. 예측 평점이 실제 평점에 비해 작을 수도 있는데 이는 내적 결과를 코사인 유사도 벡터 합으로 나누었기 때문에 생기는 현상이다.

예측 결과가 원래의 실제 평점과 얼마나 차이가 있는지 확인해본다. 예측 평가 지표는 MSE를 적용한다. 실제와 예측 평점의 차이는 기존에 평점이 부여된 데이터에 대해서만 오차 정도를 측정한다.

In [12]:
# 예측 평가 지표인 MSE를 계산하는 함수 get_mse() 생성
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 를 구함. 
def get_mse(pred, actual):
    # Ignore nonzero terms.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

# 결과 확인
print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


아이템 기반 모든 인접 이웃 MSE:  9.895354759094706


MSE를 감소시키는 방향으로 개선해나가도록 한다.

앞의 predict_rating() 함수는 많은 영화의 유사도 벡터를 이용하다 보니 상대적으로 평점 예측이 떨어졌다. 특정 영화와 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터를 적용하는 함수로 변경한다.

predict_rating_topsim() 함수는 predict_rating() 함수와 유사하지만 N인자를 가지고 있어서 TOP-N 유사도를 가지는 영화 유사도 벡터만 예측값을 계산하는데 적용한다.

In [13]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 Loop 수행. 
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰 순으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[:, col])[:-n-1:-1]]
        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T) 
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))        
    return pred

predict_rating_topsim() 함수를 이용해 계산한 예측 평점과 실제 평점의 MSE를 구해본다

In [14]:
ratings_pred = predict_rating_topsim(ratings_matrix.values , item_sim_df.values, n=20)
print('아이템 기반 인접 TOP-20 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values ))


# 계산된 예측 평점 데이터는 DataFrame으로 재생성
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index= ratings_matrix.index,
                                   columns = ratings_matrix.columns)

C:\Users\구남이\AppData\Local\Temp\ipykernel_24140\2394360531.py:11: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] = item_sim_arr[col, :][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
C:\Users\구남이\AppData\Local\Temp\ipykernel_24140\2394360531.py:12: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  pred[row, col] /= np.sum(np.abs(item_sim_arr[col, :][top_n_items]))


아이템 기반 인접 TOP-20 이웃 MSE:  3.695009387428144


이제 특정 사용자에 대해 영화를 추천해 본다. userId = 9인 사용자에 대해 영화를 추천해 본다. 먼저 9번 userID 사용자가 어떤 영화를 좋아하는지 확인해 본다. 사용자가 평점을 준 영화를 평점이 높은 순으로 나열한다.

In [15]:
user_rating_id = ratings_matrix.loc[9, :]
user_rating_id[ user_rating_id > 0].sort_values(ascending=False)[:10]

title
Adaptation (2002)                                                                 5.0
Citizen Kane (1941)                                                               5.0
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    5.0
Producers, The (1968)                                                             5.0
Lord of the Rings: The Two Towers, The (2002)                                     5.0
Lord of the Rings: The Fellowship of the Ring, The (2001)                         5.0
Back to the Future (1985)                                                         5.0
Austin Powers in Goldmember (2002)                                                5.0
Minority Report (2002)                                                            4.0
Witness (1985)                                                                    4.0
Name: 9, dtype: float64

반지의 제왕, 오스틴 파워 등 대작 영화나 어드벤처 영화, 코미디 영화 등 전반적으로 흥행성이 좋은 영화에 높은 평점을 주고 있다. 이 사용자에게 아이템 기반 협업 필터링을 통해 영화를 추천한다. 먼저 사용자가 이미 평점을 준 영화를 제외하고 추천할 수 있도록 평점을 주지 않은 영화를 리스트 객체로 반환하는 함수인 get_unseen_movies()를 생성한다.

In [16]:
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보 추출하여 Series로 반환함. 
    # 반환된 user_rating 은 영화명(title)을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[userId,:]
    
    # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 index를 추출하여 list 객체로 만듬
    already_seen = user_rating[ user_rating > 0].index.tolist()
    
    # 모든 영화명을 list 객체로 만듬. 
    movies_list = ratings_matrix.columns.tolist()
    
    # list comprehension으로 already_seen에 해당하는 movie는 movies_list에서 제외함. 
    unseen_list = [ movie for movie in movies_list if movie not in already_seen]
    
    return unseen_list

이 사용자가 영화의 평점을 주지 않은 추천 대상 영화 정보와 predict_rating_topsim()에서 추출한 사용자별 아이템 유사도에 기반한 예측 평점 데이터 세트를 이용해 최종적으로 사용자에게 영화를 추천하는 함수인 recomm_movie_by_userid()를 생성한다.

In [17]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측 평점 DataFrame에서 사용자id index와 unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬함. 
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
    return recomm_movies
    
# 사용자가 관람하지 않는 영화명 추출   
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천 
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이타를 DataFrame으로 생성. 
recomm_movies = pd.DataFrame(data=recomm_movies.values,index=recomm_movies.index,columns=['pred_score'])
recomm_movies

,pred_score
title,
Shrek (2001),0.866202
Spider-Man (2002),0.857854
"Last Samurai, The (2003)",0.817473
Indiana Jones and the Temple of Doom (1984),0.816626
"Matrix Reloaded, The (2003)",0.800990
Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001),0.765159
Gladiator (2000),0.740956
"Matrix, The (1999)",0.732693
Pirates of the Caribbean: The Curse of the Black Pearl (2003),0.689591


슈렉, 스파이더 맨, 인디아나 존스, 매트릭스 등 다양하지만 높은 흥행성을 가진 작품이 추천됐다.